# MLC v2

In [5]:
import numpy as np

# The Equation for Backpropagation
$$\begin{array}{cll}
\delta^L & = & \nabla_a C \odot \sigma'(z^L) \\ 
\delta^l & = & ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l) \\
\frac{\partial C}{\partial b^l_j} & = & \delta^l_j \\
\frac{\partial C}{\partial w^l_{jk}} & = & a_k^{l-1} \delta^l_j
\end{array}$$

# Layer Class

In [66]:
class Layer:
    def __init__(self, input_size, output_size, activation):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)
        self.activate, self.d_activate = activation
    def forward(self, x):
        self.inputs = x
        self.outputs = self.activate(x @ self.weights + self.bias)
        return self.outputs
    # Returns the delta for this layer
    def err(self, delta_p, w_p):
        #print(f"delta: {delta_p}")
        delta = (delta_p @ w_p.T) * self.d_activate(self.outputs)
        #delta = (w_p @ delta_p.T) * self.d_activate(self.outputs)
        return delta
    # Perform backpropagation, returning the weights and bias gradients for this layer
    def back(self, delta):
        #print(f"self.inputs: {self.inputs.T}")
        #print(f"delta input: {delta}")
        print(f"delta: {delta}")
        print(f"self.inputs.T: {self.inputs.T}")
        
        grad_b = delta
        grad_w =  self.inputs.T @ delta 

        return (grad_w, grad_b)

# Network 

In [67]:
class Network:
    def __init__(self, loss, d_loss):
        self.layers = []
        self.loss = loss
        self.d_loss = d_loss
    def add(self, layer):
        self.layers.append(layer)
    def fit(self, inputs, labels, epochs, learn_rate):
        for epoch in range(epochs):
            print(f"Epoch {epoch}")
            epoch_loss = 0 
            grad_w = []
            grad_b = []
            for x in enumerate(inputs):
                output = x[1]
                # forward propagate
                for layer in self.layers:
                    output = layer.forward(output)

                epoch_loss += self.loss(output, labels[x[0]])
                
                lg_w = [] # gradients are stored in reverse of layers
                lg_b = []
                # backpropagate
                #delta = self.d_loss(output, labels[x[0]]) *  self.layers[-1].d_activate(output)
                delta = self.d_loss(output, labels[x[0]]) * self.layers[-1].d_activate(output) 
                #print(f"initial delta: {delta}")
                #print(f"# of layers: {len(self.layers)}")
                for i in range(len(self.layers)-1, -1, -1):
                    #print(f"======At layer {i}")
                    gw, gb = self.layers[i].back(delta)
                    lg_w.append(gw)
                    lg_b.append(gb)
                    if i-1 >= 0:
                        delta = self.layers[i-1].err(delta, layer.weights)
                grad_w.append(lg_w)
                grad_b.append(lg_b)

            # Sum and average
            #grad_w = np.add.reduce(grad_w) / len(inputs)
            #grad_b = np.add.reduce(grad_b) / len(inputs)
            #print(f"grad_b: {grad_b}")
            grad_b_avg = []
            for i in range(len(grad_b[0])):
                b0 = grad_b[0][i]
                for j in range(1, len(grad_b)):
                    #print(f"+=+= {grad_b[j][i]}")
                    b0 += grad_b[j][i]
                grad_b_avg.append(b0 / len(grad_b))

            #print(f"grad_w: {grad_w}")
            grad_w_avg = []
            for i in range(len(grad_w[0])):
                w0 = grad_w[0][i]
                for j in range(1, len(grad_w)):
                    #print(f"+=+= {grad_w[j][i]}")
                    w0 += grad_w[j][i]
                #print(f"number of grad_ws: {len(grad_w)}")
                grad_w_avg.append(w0 / len(grad_w))
            #print(f"grad_w_avg: {grad_w_avg}")
                
            
            #print(f"grad_w: {grad_w}")
            #print(f"grad_b: {grad_b}")
            for i in range(len(self.layers)-1, -1, -1):
                layer = self.layers[i]
                #print(f"layer weights: {layer.weights}")
                #print(f"grad_w_avg: {grad_w_avg}")
                layer.weights -= learn_rate * grad_w_avg[-i+1]
                layer.bias -= learn_rate * grad_b_avg[-i+1]
            
            print(f"loss: {epoch_loss / len(inputs)}")
            

# Activator functions

In [68]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))
    
def d_sigmoid(x: np.ndarray) -> np.ndarray:
    return np.exp(-x) / ((1 + np.exp(-x))**2)

# Loss functions

In [69]:
def mse(dy, y):
    return np.mean(np.power(dy-y, 2));

def mse_prime(dy, y):
    return 2*(dy-y)/dy.size;

def tanh(x):
    return np.tanh(x);

def tanh_prime(x):
    return 1-np.tanh(x)**2;

# Example

In [70]:
net = Network(mse, mse_prime)
activator = (tanh, tanh_prime)

net.add(Layer(2, 3, activator))
net.add(Layer(3, 1, activator))

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

net.fit(x_train, y_train, epochs=1000, learn_rate=0.1)


Epoch 0
delta: [[0.89493526]]
self.inputs.T: [[0.1870139 ]
 [0.26806482]
 [0.21153731]]
delta: [[0.61318354 0.54903254 0.52561852]]
self.inputs.T: [[0]
 [0]]
delta: [[-0.0318519]]
self.inputs.T: [[0.73033181]
 [0.75267917]
 [0.73398564]]
delta: [[-0.01381841 -0.01247315 -0.01190545]]
self.inputs.T: [[0]
 [1]]
delta: [[-0.02807496]]
self.inputs.T: [[0.76372822]
 [0.83508621]
 [0.70652251]]
delta: [[-0.01167536 -0.0098603  -0.01085547]]
self.inputs.T: [[1]
 [0]]
delta: [[0.8469422]]
self.inputs.T: [[0.94083805]
 [0.95700485]
 [0.92207991]]
delta: [[0.27571176 0.24993169 0.24526706]]
self.inputs.T: [[1]
 [1]]
loss: 0.3834927254382191
Epoch 1
delta: [[0.88991014]]
self.inputs.T: [[0.16610262]
 [0.24995225]
 [0.19360226]]
delta: [[0.59431028 0.53000686 0.50655664]]
self.inputs.T: [[0]
 [0]]
delta: [[-0.04068526]]
self.inputs.T: [[0.71693306]
 [0.7414783 ]
 [0.72246229]]
delta: [[-0.0173644  -0.01554472 -0.0148444 ]]
self.inputs.T: [[0]
 [1]]
delta: [[-0.03580435]]
self.inputs.T: [[0.7517274